# Part B: Invasion Percolation

The next percolation algorithm to be demonstrated is known as ``Invasion Percolation``. Instead of identifying connected clusters and invading them all in one go, as ``Ordinary Percolation`` does, this algorithm progresses one invasion step at a time. This is a more dynamic process and better simulates scenarios where instead of controlling the pressure at the network boundaries something else such as mass flow rate is controlled as the pressure is allowed to fluctuate up and down in order to meet the lowest available entry pressure for the growing cluster(s).

In [1]:
import sys
import numpy as np
import porespy as ps
import openpnm as op
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import interact, IntSlider, widgets, interactive
from openpnm.topotools import trim
ps.visualization.set_mpl_style()
mpl.rcParams["image.interpolation"] = "None"
np.random.seed(10)
ws = op.Workspace()
ws.settings["loglevel"] = 40

In order to also showcase some other network generation options we first start with a small 2D network with ``StickAndBall`` geometry.

In [2]:
spacing = 2.5e-5
net = op.network.Cubic([20, 20, 1], spacing=spacing)
geo = op.geometry.StickAndBall(network=net, pores=net.Ps, throats=net.Ts)

We then trim all the surface pores to obtain disctint sets of boundary edge pores.

In [3]:
net.labels()
net.num_throats('surface')
trim(network=net, throats=net.throats('surface'))
h = net.check_network_health()
trim(network=net, pores=h['trim_pores'])

Then we use a function from our porespy package to generate a tomography style image of the abstract network providing the number of pixels in each dimension.

In [4]:
im = ps.io.openpnm_to_im(net, max_dim=1000)

  0%|          | 0/396 [00:00<?, ?it/s]

  0%|          | 0/684 [00:00<?, ?it/s]

In [5]:
im.shape

(1000, 1000, 50)

This creates a 3D image but we can crop it to get the central slice in 2D for visualization.

In [6]:
fig, ax = plt.subplots(figsize=(4, 4))
plt.imshow(im[25:-25, 25:-25, 25].T)
crop = im[25:-25, 25:-25, :]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next the snow algorithm is used to do network extraction on the tomography style image. Of course if you have your own tomogrpahy image this can be used instead.

In [7]:
snow_out = ps.networks.snow2(phases=crop > 0, voxel_size=4e-7)

Extracting pore and throat properties:   0%|          | 0/491 [00:00<?, ?it/s]

In [8]:
snow_out.regions.shape

(956, 956, 56)

The SNOW algorithm provides a labelled region image containing the pore index. As zero is used for the background it is actually the pore index + 1 because python references arrays with first element as zero and we do not explicitly store the pore index.

In [9]:
fig, ax = plt.subplots(figsize=(4, 4))
reg = snow_out.regions.astype(float) - 1
reg[reg == -1] = np.nan
region_slice = snow_out.regions[:, :, 28] - 1
mask = region_slice >= 0
plt.imshow(region_slice.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now our new network is extracted we can fill a network object with all the properties and begin simulation.

In [10]:
proj = op.io.PoreSpy.import_data(snow_out.network)
net = proj.network
# Add 'pore.left' label for convenience
net['pore.left'] = net['pore.xmin']
geo = proj.geometries()['geo_01']
# TODO: add some text and explain why we need it
import openpnm.models.geometry as gmods
geo.add_model(propname='throat.hydraulic_size_factors',
              model=gmods.hydraulic_size_factors.cones_and_cylinders)
geo.add_model(propname='throat.diffusive_size_factors',
              model=gmods.diffusive_size_factors.cones_and_cylinders)

A helper function is defined for plotting a particular data set.

In [11]:
def gridify(data):
    data = data.astype(float)
    out_im = np.ones(region_slice.shape, dtype=float) * -1
    out_im[mask] = data[region_slice[mask]]
    out_im[~mask] = np.nan
    return out_im

In [12]:
fig, ax = plt.subplots(figsize=(4, 4))
out = gridify(net['pore.diameter'])
plt.imshow(out.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Again, stadard physics is used to define the capillary entry pressures. And these are shown as a histogram for all the throats in the network.

In [13]:
water = op.phases.Water(network=net)
phys = op.physics.Basic(network=net, geometry=geo, phase=water)

In [14]:
fig, ax = plt.subplots(figsize=[4, 4])
ax.hist(phys['throat.entry_pressure'], bins=10);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next, the algorithm is defined and run with no arguments or outlets defined. This will proceed step by step assessing which pores are currently invaded (i.e. inlets first), which throats connect to an uninvaded pore and of these, which throat has the lowest capillary entry pressure for invasion. Invasion then proceeds along the path of least capillary resistance.

In [15]:
alg_ip = op.algorithms.InvasionPercolation(network=net)
alg_ip.setup(phase=water)
alg_ip.set_inlets(pores=net.pores('left'))
alg_ip.run()
fig, ax = plt.subplots(figsize=(4, 4))
out = gridify(alg_ip['pore.invasion_sequence'])
plt.imshow(out.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Using the slider below we can interactively plot the saturation at each invasion step (this works best using the left and right arrow keys).

In [16]:
max_seq = alg_ip['pore.invasion_sequence'].max()

fig, ax = plt.subplots(figsize=(4, 4))
data = alg_ip['pore.invasion_sequence'] < 100
obj = ax.imshow(gridify(data).T)

@widgets.interact(seq=(0, max_seq, 1))
def plot_invasion(seq):
    data = alg_ip['pore.invasion_sequence'] < seq
    obj.set_data(gridify(data));
    fig.canvas.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=385, description='seq', max=771), Output()), _dom_classes=('widget-inter…

As with ``Ordinary Percolation`` we can plot a drainage or intrusion curve but this time the capillary pressure is plotted from one step to the next as a continuous process with dynamic pressure boundary conditions and so is allowed to increase and decrease to meet the next lowest entry pressure for the invading cluster.

In [17]:
fig, ax = plt.subplots(figsize=(4, 4))
alg_ip.plot_intrusion_curve(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We can compare the results of the two algorithms and see that the pressure envelope, i.e. maximum pressure reached historically by the invasion process is the same as the ordinary percolation value.

In [18]:
fig, ax = plt.subplots(figsize=(4, 4))
alg_op = op.algorithms.OrdinaryPercolation(network=net, phase=water)
alg_op.set_inlets(net.pores('left'))
alg_op.setup(pore_volume='pore.volume', throat_volume='throat.volume')
alg_op.run(points=1000)
alg_op.plot_intrusion_curve(ax=ax)
alg_ip.plot_intrusion_curve(ax=ax)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

An additional feature of the algorithm is the ability to identify where the defending phase becomes trapped. Whether this happens in reality in-fact relies on the connectivity of the defending phase and whether it can reside in the invaded pores as thin wetting films. If not then the defending phase is completely pushed out of a pore when invaded and it can become isolated and trapped when encircled by the invading phase. OpenPNM actually calculates this trapping as a post-process, employing some clever logic described by [Masson 2016](http://dx.doi.org/10.1016/j.cageo.2016.02.003).

In [19]:
alg_ip_t = op.algorithms.InvasionPercolation(network=net)
alg_ip_t.setup(phase=water)
alg_ip_t.set_inlets(pores=net.pores('left'))
alg_ip_t.run()
alg_ip_t.apply_trapping(outlets=net.pores(['boundary']))
fig, ax = plt.subplots(figsize=(4, 4))
out = gridify(alg_ip_t['pore.trapped'])
plt.imshow(out.T);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Here a reasonable fraction of the pore space is not invaded due to trapping of the defending phase. Generally this fraction will be lower in truly 3D networks as there are more routes out of the network because pores have higher connectivity. Also, typically if a defending phase is considered to be wetting then film flow is assumed to allow residual defending phase to escape. However, we can show the differences on one plot with and without trapping below.

In [20]:
fig, ax = plt.subplots(figsize=(4, 4))
alg_ip.plot_intrusion_curve(ax=ax)
alg_ip_t.plot_intrusion_curve(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …